In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import numpy as np
from LanguageModels.Word2Vec import Word2Vec
from Preprocessing.LemmatizerPreprocessor import LemmatizerPreprocessor
from Preprocessing.DataLoader import DataLoader
from sklearn.model_selection import train_test_split
import random
import tqdm

In [5]:
data = DataLoader('../data/EMNLP2020.csv').load()
lp = LemmatizerPreprocessor()
w2v = Word2Vec(path='../data/glove.6B/glove.6B.200d.txt')

In [6]:
# multiclass case
data, label = w2v.featurize(data, lp, mode='multiclass')
print(len(data))
print(len(label))
label = label.ravel()

1450
1450


In [9]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42, stratify=label)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1160, 200) (290, 200) (1160,) (290,)


In [24]:
np.unique(y_train)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [135]:
class FeedForward(nn.Module):
    def __init__(self, dim_i, dim_h, dim_o):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(dim_i, dim_h)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()   
        self.linear2 = nn.Linear(dim_h, 100)
        self.linear3 = nn.Linear(100, dim_o)
        self.logsoftmax = nn.LogSoftmax(dim=0)
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.2)
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.dropout1(self.relu(out))
        out = self.linear2(out)
        out = self.dropout2(self.relu(out))
        out = self.linear3(out)
        output = self.logsoftmax(out)
        return output
    
num_classes  = 11
num_hidden   = 500
num_features = 200

ffnn = FeedForward(num_features, num_hidden, num_classes)
optimizer = optim.SGD(ffnn.parameters(), lr=0.001, momentum=0.9)

In [139]:
for epoch in range(500):
    total_loss = 0.0
    shuffled_i = list(range(0,len(y_train)))
    random.shuffle(shuffled_i)
    
    for i in shuffled_i:
        x = torch.from_numpy(X_train[i]).float()
        y_onehot = torch.zeros(num_classes)
        y_onehot[y_train[i]-1] = 1

        ffnn.zero_grad()
        logProbs = ffnn.forward(x)
        loss = torch.neg(logProbs).dot(y_onehot)
        total_loss += loss
        
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:    
      print("loss on epoch %i: %f" % (epoch, total_loss))

loss on epoch 0: 788.932251
loss on epoch 10: 779.399597
loss on epoch 20: 706.478577
loss on epoch 30: 640.999207
loss on epoch 40: 670.406128
loss on epoch 50: 571.681824
loss on epoch 60: 527.598938
loss on epoch 70: 517.100647
loss on epoch 80: 446.766144
loss on epoch 90: 430.587891
loss on epoch 100: 387.150879
loss on epoch 110: 392.147644
loss on epoch 120: 369.389984
loss on epoch 130: 362.253418
loss on epoch 140: 269.341736
loss on epoch 150: 299.595825
loss on epoch 160: 282.319580
loss on epoch 170: 244.171799
loss on epoch 180: 251.633331
loss on epoch 190: 288.169037
loss on epoch 200: 219.008392
loss on epoch 210: 207.348801
loss on epoch 220: 175.197266
loss on epoch 230: 196.610489
loss on epoch 240: 194.478577
loss on epoch 250: 103.167023
loss on epoch 260: 142.024399
loss on epoch 270: 152.400650
loss on epoch 280: 153.823532
loss on epoch 290: 119.545250
loss on epoch 300: 105.711578
loss on epoch 310: 161.403122
loss on epoch 320: 92.535728
loss on epoch 330: 96.

In [140]:
# Evaluate on the training set:
num_errors = 0
for i in range(len(y_train)):
    x = torch.from_numpy(X_train[i]).float()
    y = y_train[i]
    logProbs = ffnn.forward(x)
    prediction = torch.argmax(logProbs) + 1
    if y != prediction:
        num_errors += 1
        
print("number of errors: {} out of {}".format(num_errors, len(y_train)))
print('Accuracy: {}'.format(((len(y_train)-num_errors)/len(y_train)) * 100))

number of errors: 37 out of 1160
Accuracy: 96.8103448275862


In [141]:
# Evaluate on the testing set:
num_errors = 0
for i in range(len(y_test)):
    x = torch.from_numpy(X_test[i]).float()
    y = y_test[i]
    logProbs = ffnn.forward(x)
    prediction = torch.argmax(logProbs) + 1
    if y != prediction:
        num_errors += 1
        
print("number of errors: {} out of {}".format(num_errors, len(y_test)))
print('Accuracy: {}'.format(((len(y_test)-num_errors)/len(y_test)) * 100))

number of errors: 112 out of 290
Accuracy: 61.37931034482759
